In [ ]:
!pip install sqlalchemy
!pip install ipython-sql
!pip intsall pyodbc

In [ ]:
from sqlalchemy import create_engine
import os
from urllib.parse import quote_plus

username = os.environ.get('MSSQLUSER')
password = os.environ.get('MSSQLPASS')
server_name = os.environ.get('MSSQLSERVER')
database_name = "AdventureWorksDW2022"
DSN = "MyDSN"

quoted_password = quote_plus(password)

# Construct the connection URL
connection_url = f"mssql+pyodbc://{username}:{quoted_password}@{server_name}/{database_name}?driver=ODBC Driver 17 for SQL Server"
# connection_url2 = f"mssql+pyodbc://sa:f{username}@{DSN}"

engine = create_engine(connection_url)
connection = engine.connect()

In [ ]:
from sqlalchemy import inspect
inspector = inspect(engine)
for table in inspector.get_table_names():
    print(table, end=', ')

AdventureWorksDWBuildVersion, DatabaseLog, DimAccount, DimCurrency, DimCustomer, DimDate, DimDepartmentGroup, DimEmployee, DimGeography, DimOrganization, DimProduct, DimProductCategory, DimProductSubcategory, DimPromotion, DimReseller, DimSalesReason, DimSalesTerritory, DimScenario, FactAdditionalInternationalProductDescription, FactCallCenter, FactCurrencyRate, FactFinance, FactInternetSales, FactInternetSalesReason, FactProductInventory, FactResellerSales, FactSalesQuota, FactSurveyResponse, NewFactCurrencyRate, ProspectiveBuyer, sysdiagrams, 

In [ ]:
import pandas as pd

### Retrieve a list of all products and their corresponding categories.

In [ ]:
result = connection.execute('select * from DimProduct')
print(result.keys())

['ProductKey', 'ProductAlternateKey', 'ProductSubcategoryKey', 'WeightUnitMeasureCode', 'SizeUnitMeasureCode', 'EnglishProductName', 'SpanishProductName', 'FrenchProductName', 'StandardCost', 'FinishedGoodsFlag', 'Color', 'SafetyStockLevel', 'ReorderPoint', 'ListPrice', 'Size', 'SizeRange', 'Weight', 'DaysToManufacture', 'ProductLine', 'DealerPrice', 'Class', 'Style', 'ModelName', 'LargePhoto', 'EnglishDescription', 'FrenchDescription', 'ChineseDescription', 'ArabicDescription', 'HebrewDescription', 'ThaiDescription', 'GermanDescription', 'JapaneseDescription', 'TurkishDescription', 'StartDate', 'EndDate', 'Status']


In [ ]:
result = connection.execute('select * from DimProductCategory')
print(result.keys())

['ProductCategoryKey', 'ProductCategoryAlternateKey', 'EnglishProductCategoryName', 'SpanishProductCategoryName', 'FrenchProductCategoryName']


In [ ]:
result = connection.execute('select * from DimProductSubcategory')
print(result.keys())

['ProductSubcategoryKey', 'ProductSubcategoryAlternateKey', 'EnglishProductSubcategoryName', 'SpanishProductSubcategoryName', 'FrenchProductSubcategoryName', 'ProductCategoryKey']


In [ ]:
query = """
select P.ProductKey, P.EnglishProductName, PC.EnglishProductCategoryName, PSC.EnglishProductSubcategoryName
from DimProduct P
left join DimProductSubcategory PSC on P.ProductSubcategoryKey=PSC.ProductSubcategoryKey
left join DimProductCategory PC on PSC.ProductCategoryKey=PC.ProductCategoryKey;
"""
result = connection.execute(query)
rows = result.fetchall()
df = pd.DataFrame(rows, columns=result.keys())
df.head()

,ProductKey,EnglishProductName,EnglishProductCategoryName,EnglishProductSubcategoryName
0,1,Adjustable Race,None,None
1,2,Bearing Ball,None,None
2,3,BB Ball Bearing,None,None
3,4,Headset Ball Bearings,None,None
4,5,Blade,None,None


In [ ]:
df

,ProductKey,EnglishProductName,EnglishProductCategoryName,EnglishProductSubcategoryName
0,1,Adjustable Race,None,None
1,2,Bearing Ball,None,None
2,3,BB Ball Bearing,None,None
3,4,Headset Ball Bearings,None,None
4,5,Blade,None,None
...,...,...,...,...
601,602,ML Bottom Bracket,Components,Bottom Brackets
602,603,HL Bottom Bracket,Components,Bottom Brackets
603,604,"Road-750 Black, 44",Bikes,Road Bikes
604,605,"Road-750 Black, 48",Bikes,Road Bikes


### Calculate the customer churn rate for the year 2014. Churn rate is the percentage of customers who made a purchase in the previous year but didn't make any purchases in the current year.

In [ ]:
result = connection.execute('select * from DimCustomer')
print(result.keys())

['CustomerKey', 'GeographyKey', 'CustomerAlternateKey', 'Title', 'FirstName', 'MiddleName', 'LastName', 'NameStyle', 'BirthDate', 'MaritalStatus', 'Suffix', 'Gender', 'EmailAddress', 'YearlyIncome', 'TotalChildren', 'NumberChildrenAtHome', 'EnglishEducation', 'SpanishEducation', 'FrenchEducation', 'EnglishOccupation', 'SpanishOccupation', 'FrenchOccupation', 'HouseOwnerFlag', 'NumberCarsOwned', 'AddressLine1', 'AddressLine2', 'Phone', 'DateFirstPurchase', 'CommuteDistance']


In [ ]:
result = connection.execute('select * from DimDate')
print(result.keys())

['DateKey', 'FullDateAlternateKey', 'DayNumberOfWeek', 'EnglishDayNameOfWeek', 'SpanishDayNameOfWeek', 'FrenchDayNameOfWeek', 'DayNumberOfMonth', 'DayNumberOfYear', 'WeekNumberOfYear', 'EnglishMonthName', 'SpanishMonthName', 'FrenchMonthName', 'MonthNumberOfYear', 'CalendarQuarter', 'CalendarYear', 'CalendarSemester', 'FiscalQuarter', 'FiscalYear', 'FiscalSemester']


In [ ]:
result = connection.execute('select * from FactInternetSales')
print(result.keys())

['ProductKey', 'OrderDateKey', 'DueDateKey', 'ShipDateKey', 'CustomerKey', 'PromotionKey', 'CurrencyKey', 'SalesTerritoryKey', 'SalesOrderNumber', 'SalesOrderLineNumber', 'RevisionNumber', 'OrderQuantity', 'UnitPrice', 'ExtendedAmount', 'UnitPriceDiscountPct', 'DiscountAmount', 'ProductStandardCost', 'TotalProductCost', 'SalesAmount', 'TaxAmt', 'Freight', 'CarrierTrackingNumber', 'CustomerPONumber', 'OrderDate', 'DueDate', 'ShipDate']


In [ ]:
query = '''
    SELECT Customer.CustomerKey, SalesCustomer.CustomerKey
    FROM DimCustomer Customer
    LEFT JOIN (
        select distinct CustomerKey
        from FactInternetSales Sales
        join DimDate Date on Sales.OrderDateKey=Date.DateKey
        where Date.CalendarYear=2014
    ) as SalesCustomer
    on Customer.CustomerKey = SalesCustomer.CustomerKey
    where Customer.CustomerKey is Null

'''
result = connection.execute(query)
rows = result.fetchall()
df = pd.DataFrame(rows, columns=result.keys())
df

,CustomerKey,CustomerKey


In [ ]:
query = '''
        select distinct CustomerKey
        from FactInternetSales Sales
        join DimDate Date on Sales.OrderDateKey=Date.DateKey
        where Date.CalendarYear<2014
'''
result = connection.execute(query)
rows = result.fetchall()
df = pd.DataFrame(rows, columns=result.keys())
df

,CustomerKey
0,15652
1,14324
2,18569
3,27059
4,22814
...,...
17973,27050
17974,14066
17975,18935
17976,12984


In [ ]:
query = '''
    SELECT distinct Customer.CustomerKey
    FROM DimCustomer Customer
    LEFT JOIN (
        select distinct CustomerKey
        from FactInternetSales Sales
        join DimDate Date on Sales.OrderDateKey=Date.DateKey
        where Date.CalendarYear=2014
    ) as SalesCustomer
    on Customer.CustomerKey = SalesCustomer.CustomerKey
    where SalesCustomer.CustomerKey is Null
'''
result = connection.execute(query)
rows = result.fetchall()
df = pd.DataFrame(rows, columns=result.keys())
df

,CustomerKey
0,11000
1,11001
2,11002
3,11003
4,11004
...,...
17645,29479
17646,29480
17647,29481
17648,29482


In [ ]:
import pandas as pd

query = '''
WITH CustomersWhoPurchasedBefore2014 AS (
    SELECT DISTINCT CustomerKey
    FROM FactInternetSales Sales
    JOIN DimDate Date ON Sales.OrderDateKey = Date.DateKey
    WHERE Date.CalendarYear < 2014
),

CustomersWhoDidNotPurchaseIn2014 AS (
    SELECT DISTINCT Customer.CustomerKey
    FROM DimCustomer Customer
    LEFT JOIN (
        SELECT DISTINCT CustomerKey
        FROM FactInternetSales Sales
        JOIN DimDate Date ON Sales.OrderDateKey = Date.DateKey
        WHERE Date.CalendarYear = 2014
    ) AS SalesCustomer ON Customer.CustomerKey = SalesCustomer.CustomerKey
    WHERE SalesCustomer.CustomerKey IS NULL
)

SELECT
    COUNT(DISTINCT cd.CustomerKey) AS ChurnedCustomers,
    COUNT(DISTINCT cp.CustomerKey) AS TotalCustomers,
    (COUNT(DISTINCT cd.CustomerKey) * 100.0) / COUNT(DISTINCT cp.CustomerKey) AS ChurnRate
FROM CustomersWhoPurchasedBefore2014 CP
LEFT JOIN CustomersWhoDidNotPurchaseIn2014 CD ON CP.CustomerKey = CD.CustomerKey;
'''

# Assuming you have a valid database connection named 'connection'
result = connection.execute(query)
rows = result.fetchall()
df = pd.DataFrame(rows, columns=result.keys())
df


,ChurnedCustomers,TotalCustomers,ChurnRate
0,17650,17978,98.175547891867
